# Comparing previous SID convergence with TSSOS #

In [1]:
# using packages
using DynamicPolynomials
using LinearAlgebra
using HDF5
using TSSOS
using Dates

Declare unknown symbolic matrices to build dynamic equations

In [2]:
# unknown Hermitian 2 x 2 matrix
@polyvar x[1:4]
H = [ 1.0 * x[1]        x[3] + im * x[4]
      x[3] - im * x[4]  x[2]             ]

# unknown arbitrary complex 2 x 2 matrix
@polyvar a[1:2, 1:2]
@polyvar b[1:2, 1:2]
A = 1.0 * a + im * b

2×2 Matrix{Polynomial{true, ComplexF64}}:
 a₁₋₁ + (0.0+1.0im)b₁₋₁  a₁₋₂ + (0.0+1.0im)b₁₋₂
 a₂₋₁ + (0.0+1.0im)b₂₋₁  a₂₋₂ + (0.0+1.0im)b₂₋₂

Function to build left hand side of the Lindblad equation

In [3]:
function lindblad_rhs(rho)
    return -1.0 * im * (H * rho - rho * H) + A * rho * A' - (A' * A  * rho + rho * A' * A) / 2
end

lindblad_rhs (generic function with 1 method)

Function to calculate square of Frobenius norm of the matrix

In [4]:
function frobenius_norm2(m)
    return tr(m * m')
end

frobenius_norm2 (generic function with 1 method)

Function to get seeds from the HDF5 data file for specific noise level <em> w </em>

In [5]:
function get_seeds(file_name, w)
    h5open(file_name, "r") do file
        return keys(file[string(w)])
    end
end    

get_seeds (generic function with 1 method)

In [6]:
function get_noise_levels(file_name)
    h5open(file_name, "r") do file
        return keys(file)
    end
end

get_noise_levels (generic function with 1 method)

Function to find the best(minimum) value of the objective polynomial

In [51]:
function get_rho_series(file_name, w, seed)
    h5open(file_name, "r") do file
        dset = file[string(w)][string(seed)]["rho"]
        return(read(dset))
    end
end

function opt_tssos(ρ_series)
    
    # objective for optimization 
      
    obj = sum(
        frobenius_norm2(
            ρ_series[:, :, t] - ρ_series[:, :, t - 1] 
            - dt * lindblad_rhs(0.5 * (ρ_series[:, :, t] + ρ_series[:, :, t - 1]))
        ) for t in 2:size(ρ_series,3)
    )

    
    obj_re = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    
    # execute the first level of the TSSOS hierarchy

    opt,sol,data = tssos_first(obj_re, variables(obj_re))
    
    # execute higher levels of the TSSOS hierarchy
    
    iter = 1
    best_opt = opt

    while !isnothing(opt)
        opt,sol,data = tssos_higher!(data)
        if !isnothing(opt) & (iter < 5)
            iter += 1
            if abs(opt) < abs(best_opt)
                best_opt = opt
            end
        end
    end
    
    return(best_opt, iter)
    
end    

opt_tssos (generic function with 1 method)

Reading time series of the dynamics of density matrix $\rho$ from HDF file and calculating optimums with TSSOS

In [21]:
noise_levels

5-element Vector{String}:
 "0.0"
 "0.01"
 "0.02"
 "0.04"
 "0.05"

In [52]:
file_name = "time_series_lindblad_rnd_rho_2022-04-27_03-35-20.hdf5"

noise_levels = get_noise_levels(file_name)
seeds = get_seeds(file_name, "0.05")
seeds[199]

"1039283195"

In [53]:
ρ0 = get_rho_series(file_name, "0.0", "1039283195")
ρ5 = get_rho_series(file_name, "0.05", "1039283195")

ρ = ρ0-ρ5
ρ0[:,:,1]

2×2 Matrix{ComplexF64}:
 0.282078+0.0im       0.0+0.0im
      0.0+0.0im  0.717922+0.0im

In [30]:
ρ5[:,:,1]

2×2 Matrix{ComplexF64}:
   0.286027+0.0im         -0.0147395-0.00579536im
 -0.0147395+0.00579536im    0.713973+0.0im

In [31]:
ρ 

2×2×50 Array{ComplexF64, 3}:
[:, :, 1] =
 -0.00394867+0.0im          0.0147395+0.00579536im
   0.0147395-0.00579536im  0.00394867+0.0im

[:, :, 2] =
   -0.022894+0.0im        0.000755658-0.0166903im
 0.000755658+0.0166903im     0.022894+0.0im

[:, :, 3] =
 0.000329136+0.0im            0.0021572-0.00443961im
   0.0021572+0.00443961im  -0.000329136+0.0im

;;; … 

[:, :, 48] =
  0.00584158+0.0im        -0.00556756+0.0114078im
 -0.00556756-0.0114078im  -0.00584158+0.0im

[:, :, 49] =
   0.0180146+0.0im         0.000500408-0.00275692im
 0.000500408+0.00275692im   -0.0180146+0.0im

[:, :, 50] =
  0.00930099+0.0im         0.000500501-0.00275682im
 0.000500501+0.00275682im  -0.00930099+0.0im

In [64]:
file_name = "time_series_lindblad_rnd_rho_2022-04-27_03-35-20.hdf5"

noise_levels = get_noise_levels(file_name)
seeds = get_seeds(file_name, "0.05")

ρ_ser = get_rho_series(file_name, "0.0", seeds[10])

dt = 0.1

obj = sum(
        frobenius_norm2(
            ρ_ser[:, :, t] - ρ_ser[:, :, t - 1] 
            - dt * lindblad_rhs(0.5 * (ρ_ser[:, :, t] + ρ_ser[:, :, t - 1]))
        ) for t in 2:size(ρ_ser,3)
    )

obj_re = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))

opt,sol,data = tssos_first(obj_re, variables(obj_re))

************************TSSOS************************
TSSOS is launching...
Starting to compute the block structure...
------------------------------------------------------
The sizes of PSD blocks:
[41, 8]
[1, 1]
------------------------------------------------------
Obtained the block structure. The maximal size of blocks is 41.
Assembling the SDP...
There are 525 affine constraints.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 525             
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries      

(1.1051228661569545e-6, nothing, TSSOS.upop_data(12, 0, PolyVar{true}[x₁, x₂, x₃, x₄, a₁₋₁, a₂₋₁, a₁₋₂, a₂₋₂, b₁₋₁, b₂₋₁, b₁₋₂, b₂₋₂], 0.027303451427726114a₁₋₁⁴ + 0.11319187462596121a₁₋₁³a₂₋₁ - 0.14047008155731677a₁₋₁³a₁₋₂ - 0.10921380571090446a₁₋₁³a₂₋₂ - 0.04958526970021238a₁₋₁³b₂₋₁ - 0.06221819867467921a₁₋₁³b₁₋₂ + 0.21026513727140453a₁₋₁²a₂₋₁² - 0.46476629448275797a₁₋₁²a₂₋₁a₁₋₂ - 0.3668538308092393a₁₋₁²a₂₋₁a₂₋₂ + 0.04958526970021238a₁₋₁²a₂₋₁b₁₋₁ - 0.06821985019456936a₁₋₁²a₂₋₁b₁₋₂ - 0.16138873807510404a₁₋₁²a₂₋₁b₂₋₂ + 0.39174735316461756a₁₋₁²a₁₋₂² + 0.4486884516033058a₁₋₁²a₁₋₂a₂₋₂ + 0.06221819867467921a₁₋₁²a₁₋₂b₁₋₁ + 0.06821985019456936a₁₋₁²a₁₋₂b₂₋₁ + 0.06821985019456937a₁₋₁²a₁₋₂b₁₋₂ + 0.04958526970021239a₁₋₁²a₁₋₂b₂₋₂ + 0.16382070856635667a₁₋₁²a₂₋₂² + 0.16138873807510404a₁₋₁²a₂₋₂b₂₋₁ + 0.19928752499850447a₁₋₁²a₂₋₂b₁₋₂ + 0.05460690285545223a₁₋₁²b₁₋₁² + 0.11319187462596121a₁₋₁²b₁₋₁b₂₋₁ - 0.14047008155731677a₁₋₁²b₁₋₁b₁₋₂ - 0.10921380571090446a₁₋₁²b₁₋₁b₂₋₂ + 0.21026513727140453a₁₋₁²b₂₋₁² +

In [48]:
maximum(abs(x) for x in coefficients(obj5-obj_re5))

4.0245584642661925e-16

In [50]:
maximum(abs(x) for x in coefficients(obj0-obj_re0))

3.0531133177191805e-16

In [36]:
opt,sol,data = tssos_first((obj_re0 - obj_re5)^2, variables(obj_re))

************************TSSOS************************
TSSOS is launching...
Starting to compute the block structure...

LoadError: InterruptException:

In [20]:
opt,sol,data = tssos_first(obj_re, variables(obj_re))

************************TSSOS************************
TSSOS is launching...
Starting to compute the block structure...
------------------------------------------------------
The sizes of PSD blocks:
[41, 1]
[1, 8]
------------------------------------------------------
Obtained the block structure. The maximal size of blocks is 41.
Assembling the SDP...
There are 525 affine constraints.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 525             
  Cones                  : 0               
  Scalar variables       : 9               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries      

(-1.8220520112569445e-8, nothing, TSSOS.upop_data(12, 0, PolyVar{true}[x₁, x₂, x₃, x₄, a₁₋₁, a₂₋₁, a₁₋₂, a₂₋₂, b₁₋₁, b₂₋₁, b₁₋₂, b₂₋₂], 3.7484619832975725a₁₋₁⁴ - 6.689519794186905a₁₋₁³a₂₋₁ + 6.996089409787173a₁₋₁³a₁₋₂ - 14.99384793319029a₁₋₁³a₂₋₂ - 15.59427769840626a₁₋₁³b₂₋₁ - 17.414660110550624a₁₋₁³b₁₋₂ + 27.452101402206104a₁₋₁²a₂₋₁² - 32.515015385316524a₁₋₁²a₂₋₁a₁₋₂ + 20.37512899816098a₁₋₁²a₂₋₁a₂₋₂ + 15.59427769840626a₁₋₁²a₂₋₁b₁₋₁ + 10.125937629783838a₁₋₁²a₂₋₁b₁₋₂ - 48.60321550736313a₁₋₁²a₂₋₁b₂₋₂ + 34.83527282787136a₁₋₁²a₁₋₂² - 21.294837844961776a₁₋₁²a₁₋₂a₂₋₂ + 17.414660110550624a₁₋₁²a₁₋₂b₁₋₁ - 10.125937629783838a₁₋₁²a₁₋₂b₂₋₁ - 10.125937629783836a₁₋₁²a₁₋₂b₁₋₂ + 15.594277698406259a₁₋₁²a₁₋₂b₂₋₂ + 22.49077189978544a₁₋₁²a₂₋₂² + 48.60321550736313a₁₋₁²a₂₋₂b₂₋₁ + 54.064362743796195a₁₋₁²a₂₋₂b₁₋₂ + 7.496923966595145a₁₋₁²b₁₋₁² - 6.689519794186905a₁₋₁²b₁₋₁b₂₋₁ + 6.996089409787173a₁₋₁²b₁₋₁b₁₋₂ - 14.99384793319029a₁₋₁²b₁₋₁b₂₋₂ + 27.452101402206104a₁₋₁²b₂₋₁² + 53.149315806592284a₁₋₁²b₂₋₁b₁₋₂ + 20.

In [65]:
opt2,sol2,data2 = tssos_higher!(data)

Starting to compute the block structure...
No higher TSSOS hierarchy!

(nothing, nothing, TSSOS.upop_data(12, 0, PolyVar{true}[x₁, x₂, x₃, x₄, a₁₋₁, a₂₋₁, a₁₋₂, a₂₋₂, b₁₋₁, b₂₋₁, b₁₋₂, b₂₋₂], 0.027303451427726114a₁₋₁⁴ + 0.11319187462596121a₁₋₁³a₂₋₁ - 0.14047008155731677a₁₋₁³a₁₋₂ - 0.10921380571090446a₁₋₁³a₂₋₂ - 0.04958526970021238a₁₋₁³b₂₋₁ - 0.06221819867467921a₁₋₁³b₁₋₂ + 0.21026513727140453a₁₋₁²a₂₋₁² - 0.46476629448275797a₁₋₁²a₂₋₁a₁₋₂ - 0.3668538308092393a₁₋₁²a₂₋₁a₂₋₂ + 0.04958526970021238a₁₋₁²a₂₋₁b₁₋₁ - 0.06821985019456936a₁₋₁²a₂₋₁b₁₋₂ - 0.16138873807510404a₁₋₁²a₂₋₁b₂₋₂ + 0.39174735316461756a₁₋₁²a₁₋₂² + 0.4486884516033058a₁₋₁²a₁₋₂a₂₋₂ + 0.06221819867467921a₁₋₁²a₁₋₂b₁₋₁ + 0.06821985019456936a₁₋₁²a₁₋₂b₂₋₁ + 0.06821985019456937a₁₋₁²a₁₋₂b₁₋₂ + 0.04958526970021239a₁₋₁²a₁₋₂b₂₋₂ + 0.16382070856635667a₁₋₁²a₂₋₂² + 0.16138873807510404a₁₋₁²a₂₋₂b₂₋₁ + 0.19928752499850447a₁₋₁²a₂₋₂b₁₋₂ + 0.05460690285545223a₁₋₁²b₁₋₁² + 0.11319187462596121a₁₋₁²b₁₋₁b₂₋₁ - 0.14047008155731677a₁₋₁²b₁₋₁b₁₋₂ - 0.10921380571090446a₁₋₁²b₁₋₁b₂₋₂ + 0.21026513727140453a₁₋₁²b₂₋₁² + 0.31463210466

In [15]:
opt3,sol3,data3 = tssos_higher!(data2)

Starting to compute the block structure...
No higher TSSOS hierarchy!


(nothing, nothing, TSSOS.upop_data(12, 0, PolyVar{true}[x₁, x₂, x₃, x₄, a₁₋₁, a₂₋₁, a₁₋₂, a₂₋₂, b₁₋₁, b₂₋₁, b₁₋₂, b₂₋₂], 3.9013984560897357a₁₋₁⁴ - 6.8275060551687385a₁₋₁³a₂₋₁ + 7.149253186027157a₁₋₁³a₁₋₂ - 15.605593824358943a₁₋₁³a₂₋₂ - 15.836285113303198a₁₋₁³b₂₋₁ - 17.742161423756077a₁₋₁³b₁₋₂ + 27.631079865289387a₁₋₁²a₂₋₁² - 31.909351963306776a₁₋₁²a₂₋₁a₁₋₂ + 20.804265296364633a₁₋₁²a₂₋₁a₂₋₂ + 15.836285113303198a₁₋₁²a₂₋₁b₁₋₁ + 10.560669369472265a₁₋₁²a₂₋₁b₁₋₂ - 49.41473165036247a₁₋₁²a₂₋₁b₂₋₂ + 35.25510180274852a₁₋₁²a₁₋₂² - 21.769506688939884a₁₋₁²a₁₋₂a₂₋₂ + 17.742161423756077a₁₋₁²a₁₋₂b₁₋₁ - 10.560669369472265a₁₋₁²a₁₋₂b₂₋₁ - 10.560669369472265a₁₋₁²a₁₋₂b₁₋₂ + 15.8362851133032a₁₋₁²a₁₋₂b₂₋₂ + 23.40839073653841a₁₋₁²a₂₋₂² + 49.41473165036247a₁₋₁²a₂₋₂b₂₋₁ + 55.13236058172113a₁₋₁²a₂₋₂b₁₋₂ + 7.802796912179471a₁₋₁²b₁₋₁² - 6.8275060551687385a₁₋₁²b₁₋₁b₂₋₁ + 7.149253186027157a₁₋₁²b₁₋₁b₁₋₂ - 15.605593824358943a₁₋₁²b₁₋₁b₂₋₂ + 27.631079865289387a₁₋₁²b₂₋₁² + 53.31132004974846a₁₋₁²b₂₋₁b₁₋₂ + 20.804265296364

In [9]:
println("Noise levels")

h5open(file_name, "r") do file
        println(keys(file))
    end

Noise levels
["0.0", "0.01", "0.02", "0.04", "0.05"]


In [10]:
w = "0.05"
seeds = get_seeds(output_file_name, w)
println(seeds)

LoadError: KeyError: key "0.05" not found

In [11]:
seed = seeds[3]

println("For w="*w*" and seed="*seed)

h5open(output_file_name, "r") do file
       println("Opminum:")
       println(read(file[w][seed]["opt"]))
       println("Inter_num:")
       println(read(file[w][seed]["iter"]))
    end

LoadError: UndefVarError: seeds not defined